In [2]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import cycler

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [3]:
data_dir = "./"
db_file = 'fundos.db'
engine = create_engine(
    "sqlite:///"+os.path.abspath(os.path.join(data_dir, db_file)))

In [7]:
sqlquery = """
select CNPJ_FUNDO, DT_COMPTC, CNPJ_FUNDO_COTA, VL_MERC_POS_FINAL, VL_PATRIM_LIQ  from cda
WHERE NOT CNPJ_FUNDO_COTA IS NULL
OR NOT VL_PATRIM_LIQ IS NULL
"""
df = pd.read_sql(sqlquery,engine)
df.columns

Index(['CNPJ_FUNDO', 'DT_COMPTC', 'CNPJ_FUNDO_COTA', 'VL_MERC_POS_FINAL',
       'VL_PATRIM_LIQ'],
      dtype='object')

In [12]:
pl = df.groupby(['CNPJ_FUNDO','DT_COMPTC'])

In [16]:
df['VL_PATRIM_LIQ'] = pl['VL_PATRIM_LIQ'].transform('max')

In [21]:
df = df.loc[~df.CNPJ_FUNDO_COTA.isna()]

In [23]:
df['peso'] = df['VL_MERC_POS_FINAL']/df['VL_PATRIM_LIQ']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
grp = df.groupby(['CNPJ_FUNDO','DT_COMPTC'])

In [38]:
r =grp['CNPJ_FUNDO_COTA'].count().groupby(level='CNPJ_FUNDO').mean().sort_values(ascending=False)
r

CNPJ_FUNDO
14.492.346/0001-69    76.421053
20.701.533/0001-08    43.723404
20.983.041/0001-52    41.000000
28.581.078/0001-66    41.000000
22.128.298/0001-52    40.692308
27.036.652/0001-32    40.470588
26.680.230/0001-32    40.200000
13.200.528/0001-56    39.600000
13.200.401/0001-37    39.588889
13.200.296/0001-36    39.533333
13.200.310/0001-00    39.511111
18.330.763/0001-66    36.200000
23.875.831/0001-02    35.218750
27.328.717/0001-13    35.210526
26.406.985/0001-43    35.071429
08.669.700/0001-85    34.444444
23.872.491/0001-66    34.312500
14.843.748/0001-60    34.166667
08.533.809/0001-90    34.166667
08.744.334/0001-81    34.008000
14.843.679/0001-95    33.933333
28.558.118/0001-59    33.923077
25.381.867/0001-65    33.500000
08.533.900/0001-06    33.421053
29.726.151/0001-03    33.250000
27.129.128/0001-06    32.736842
25.381.770/0001-52    32.708333
08.799.282/0001-40    32.518519
26.243.385/0001-01    32.400000
21.596.763/0001-17    32.342857
                        ...  

In [46]:
rank = pl['VL_PATRIM_LIQ'].max().groupby(level='CNPJ_FUNDO').max().rank()

In [57]:
r.head(10).to_frame().join(rank)

,CNPJ_FUNDO_COTA,VL_PATRIM_LIQ
CNPJ_FUNDO,,
14.492.346/0001-69,76.421053,29180.0
20.701.533/0001-08,43.723404,16310.0
20.983.041/0001-52,41.000000,18159.0
28.581.078/0001-66,41.000000,20287.0
22.128.298/0001-52,40.692308,28117.0
27.036.652/0001-32,40.470588,15414.0
26.680.230/0001-32,40.200000,12365.0
13.200.528/0001-56,39.600000,18778.0
13.200.401/0001-37,39.588889,18675.0


In [59]:
r.head(10).to_frame()

,CNPJ_FUNDO_COTA
CNPJ_FUNDO,
14.492.346/0001-69,76.421053
20.701.533/0001-08,43.723404
20.983.041/0001-52,41.000000
28.581.078/0001-66,41.000000
22.128.298/0001-52,40.692308
27.036.652/0001-32,40.470588
26.680.230/0001-32,40.200000
13.200.528/0001-56,39.600000
13.200.401/0001-37,39.588889
